# Task 3: Deep Verification Suite (Go/No-Go Checks)

## Checklist Items to Verify
1. **Retrieval-Generation Alignment:** Zero hallucinations.
2. **Semantic Retrieval Quality:** Relevant chunks.
3. **Source Traceability & API Readiness:** JSON schema matches React requirements.
4. **Prompt Obedience:** "I don't have enough information" check.
5. **Edge Cases:** Empty/Nonsense queries.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath('../src'))
from rag_pipeline import RAGPipeline
import json

rag = RAGPipeline()

## 1. API / React Readiness (Schema Check)
**Expectation:** `answer`, `sources` list with `text`, `product`, `company`, `complaint_id`.

In [ ]:
response = rag.answer_question("What are the fees for late payment?")
print(json.dumps(response, indent=2))

assert "answer" in response, "Missing 'answer' field"
assert "sources" in response, "Missing 'sources' field"
if response['sources']:
    source = response['sources'][0]
    assert "text" in source, "Missing 'text' in source"
    assert "product" in source, "Missing 'product' in source"
    assert "company" in source, "Missing 'company' in source"
    assert "complaint_id" in source, "Missing 'complaint_id' in source"
    print("✅ Schema Validation Passed")
else:
    print("⚠️ No sources returned (Schema check skipped for logic, but structure is valid)")

## 2. Hallucination & Prompt Guardrails
**Test:** Ask a question outside the dataset scope.
**Expectation:** Model refuses to answer.

In [ ]:
out_of_scope_q = "Who won the 1994 World Cup?"
response_oos = rag.answer_question(out_of_scope_q)
print(f"Answer: {response_oos['answer']}")

if "don't have enough information" in response_oos['answer'] or "not enough information" in response_oos['answer'] or "no information" in response_oos['answer'].lower():
    print("✅ Guardrail Passed (Refused to Hallucinate)")
else:
    print(f"❌ Guardrail Failed. Answered: {response_oos['answer']}")

## 3. Edge Case Handling
**Test:** Empty string.

In [ ]:
try:
    rag.answer_question("")
    print("✅ Handled empty string safely")
except Exception as e:
    print(f"❌ Crashed on empty string: {e}")